# Activation Quantization

### Import Packages

In [1]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras import layers, Sequential # type: error
from tensorflow.keras.applications.vgg16 import preprocess_input # type: error
from tensorflow.keras.layers import Conv2D, Dense # type: error
from tensorflow.keras.models import clone_model # type: error

In [2]:
# Local Packages
from ml_project_util.path import path_definition
from ml_project_util.flatten_model import flatten_condtitional
from ml_project_util.model_evaluation import model_evaluation_precise
from ml_project_util.quantization_util import quant_activations

### Variable Paths, Names, Execution Environments

In [3]:
BASE_PATH, PATH_DATASET, PATH_RAWDATA, PATH_JOINEDDATA, PATH_SAVEDMODELS = path_definition()
model_name = 'CD4_P2_FT_003_val0.0336'

### Load Float Model

In [4]:
short_name = model_name[:-10]
parent_name = model_name[:3]
filepath = f'{PATH_SAVEDMODELS}/{parent_name}/{model_name}.keras'
model = tf.keras.models.load_model(filepath)
model = flatten_condtitional(model, model_name)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 30,056,261 (114.66 MB)

 Trainable params: 7,473,665 (28.51 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

 Optimizer params: 14,947,332 (57.02 MB)

### Create New Model with Fake Quant Layers & Evaluate

In [5]:
quant_aware_model =  quant_activations(model, model_name, num_bits=8, mode_func='eval', mode='hw')

Quantization on symmetric ranges that enable shifting on interlayer scaling is applied.
Quantization range not found in C:/Programming_Files/JupyterVSCode/Binary_Classification_Transfer_Learning/CatsDogs/Docs_Reports/Quant/Ranges/CD4_P2_FT_003_activation_hw_range.json, recalculating.
Read activation range json from C:/Programming_Files/JupyterVSCode/Binary_Classification_Transfer_Learning/CatsDogs/Docs_Reports/Quant/Ranges/CD4_P2_FT_003_activation_sw_range.json
input_layer: min = -151.0610, max = 151.0610
block1_conv1: min = 0.0000, max = 932.9594
block1_conv2: min = 0.0000, max = 3747.8220
block2_conv1: min = 0.0000, max = 7530.5459
block2_conv2: min = 0.0000, max = 12263.2021
block3_conv1: min = 0.0000, max = 18402.4238
block3_conv2: min = 0.0000, max = 18064.2148
block3_conv3: min = 0.0000, max = 21941.4180
block4_conv1: min = 0.0000, max = 14641.2715
block4_conv2: min = 0.0000, max = 6983.7476
block4_conv3: min = 0.0000, max = 4462.6309
block5_conv1: min = 0.0000, max = 2829.6855
b

c:\Users\Aris_Work\anaconda3\envs\EnvPy3_12\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Batch Number: 0
Batch Number: 1
Batch Number: 2
Batch Number: 3
Batch Number: 4
Batch Number: 5
Batch Number: 6
Batch Number: 7
Batch Number: 8
Batch Number: 9
Batch Number: 10
Batch Number: 11
Batch Number: 12
Batch Number: 13
Batch Number: 14
Batch Number: 15
Batch Number: 16
Batch Number: 17
Batch Number: 18
Batch Number: 19
Batch Number: 20
Batch Number: 21
Batch Number: 22
Batch Number: 23
Batch Number: 24
Batch Number: 25
Batch Number: 26
Batch Number: 27
Batch Number: 28
Batch Number: 29
Batch Number: 30
Batch Number: 31
Batch Number: 32
Batch Number: 33
Batch Number: 34
Batch Number: 35
Batch Number: 36
Batch Number: 37
Batch Number: 38
Batch Number: 39
Batch Number: 40
Batch Number: 41
Batch Number: 42
Batch Number: 43
Batch Number: 44
Batch Number: 45
Batch Number: 46
Batch Number: 47
Batch Number: 48
Batch Number: 49
Batch Number: 50
Batch Number: 51
Batch Number: 52
Batch Number: 53
Batch Number: 54
Batch Number: 55
Batch Number: 56
Batch Number: 57
Batch Number: 58
Batch N

In [ ]:
model_evaluation_precise(quant_aware_model)

---

In [9]:
# Float range dictionary path
range_name = model_name[:-10]
range_dict_path = f'{BASE_PATH}/Docs_Reports/Quant/Ranges/{range_name}_activation_range.json'

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense

# Custom FakeQuantLayer simulates quantization but preserves shape
class FakeQuantLayer(tf.keras.layers.Layer):
    def __init__(self, min_val=-6.0, max_val=6.0):
        super().__init__()
        self.min_val = min_val
        self.max_val = max_val

    def call(self, inputs):
        return tf.quantization.fake_quant_with_min_max_vars(inputs, min=self.min_val, max=self.max_val)
    
class SymmetricFakeQuantLayer(tf.keras.layers.Layer):
    def __init__(self, max_abs_val=6.0, num_bits=8, narrow_range=True, **kwargs):
        super().__init__(**kwargs)
        self.max_abs_val = max_abs_val
        self.min_val = -max_abs_val
        self.max_val = max_abs_val
        self.num_bits = num_bits
        self.narrow_range = narrow_range  # Set to True for signed int8 [-127, 127]

    def call(self, inputs):
        return tf.quantization.fake_quant_with_min_max_vars(
            inputs,
            min=self.min_val,
            max=self.max_val,
            num_bits=self.num_bits,
            narrow_range=self.narrow_range
        )

def clone_model_with_fake_quant(original_model, input_shape, range_dict):
    new_model = Sequential()
    layer_mapping = []
    quant_layers_list = list(range_dict.keys())

    # Add input layer explicitly
    new_model.add(tf.keras.Input(shape=input_shape))

    quant_layer = 0
    for layer in original_model.layers:
        config = layer.get_config()
        cloned_layer = layer.__class__.from_config(config)
        # Insert fake quant after Conv2D or Dense
        if isinstance(cloned_layer, (Conv2D, Dense)):
            tmp_min = range_dict[quant_layers_list[quant_layer]]['min']
            tmp_max = range_dict[quant_layers_list[quant_layer]]['max']
            abs_max = abs(tmp_min) if abs(tmp_min)>tmp_max else tmp_max
            #new_model.add(FakeQuantLayer(min_val=tmp_min, max_val=tmp_max))
            new_model.add(SymmetricFakeQuantLayer(max_abs_val=abs_max))
            quant_layer = quant_layer + 1
        # Clone layer from config
        new_model.add(cloned_layer)
        layer_mapping.append((layer, cloned_layer))

    # Build model by running dummy data through it
    dummy_input = tf.random.uniform((1, *input_shape))
    new_model(dummy_input)

    # Copy weights from original layers to cloned layers
    for orig_layer, cloned_layer in layer_mapping:
        if orig_layer.weights and cloned_layer.weights:
            try:
                cloned_layer.set_weights(orig_layer.get_weights())
            except ValueError as e:
                print(f"Skipping weights for layer {orig_layer.name} due to mismatch: {e}")

    new_model.build(input_shape=(None, *input_shape))  # Step 2

    dummy_input = tf.random.uniform((1, *input_shape))  # Step 3
    new_model(dummy_input)

    print("New model input shape:", new_model.input_shape)  # Step 4

    for orig_layer, cloned_layer in layer_mapping:
        try:
            cloned_layer.set_weights(orig_layer.get_weights())
        except Exception as e:
            print(f"Skipping weights for {orig_layer.name}: {e}")

    return new_model

In [10]:
input_shape = (224, 224, 3)
try:
    with open(range_dict_path, 'r') as file:
        range_dict = json.load(file)
except:
    print('No float range dictionary found!')
quant_aware_model = clone_model_with_fake_quant(model, input_shape, range_dict)
quant_aware_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
quant_aware_model.summary()


New model input shape: (None, 224, 224, 3)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ symmetric_fake_quant_layer      │ (None, 224, 224, 3)    │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_1    │ (None, 224, 224, 64)   │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_2    │ (None, 112, 112, 64)   │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_3    │ (None, 112, 112, 128)  │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_4    │ (None, 56, 56, 128)    │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_5    │ (None, 56, 56, 256)    │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_6    │ (None, 56, 56, 256)    │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_7    │ (None, 28, 28, 256)    │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ symmetric_fake_quant_layer_8    │ (None, 28, 28, 512)    │             0 │
│ (SymmetricFakeQuantLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 15,108,929 (57.64 MB)

 Trainable params: 7,473,665 (28.51 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [12]:
model_evaluation_precise(quant_aware_model)

Found 24997 files belonging to 2 classes.
Using 4999 files for validation.
Batch Number: 0
Batch Number: 1
Batch Number: 2
Batch Number: 3
Batch Number: 4
Batch Number: 5
Batch Number: 6
Batch Number: 7
Batch Number: 8
Batch Number: 9
Batch Number: 10
Batch Number: 11
Batch Number: 12
Batch Number: 13
Batch Number: 14
Batch Number: 15
Batch Number: 16
Batch Number: 17
Batch Number: 18
Batch Number: 19
Batch Number: 20
Batch Number: 21
Batch Number: 22
Batch Number: 23
Batch Number: 24
Batch Number: 25
Batch Number: 26
Batch Number: 27
Batch Number: 28
Batch Number: 29
Batch Number: 30
Batch Number: 31
Batch Number: 32
Batch Number: 33
Batch Number: 34
Batch Number: 35
Batch Number: 36
Batch Number: 37
Batch Number: 38
Batch Number: 39
Batch Number: 40
Batch Number: 41
Batch Number: 42
Batch Number: 43
Batch Number: 44
Batch Number: 45
Batch Number: 46
Batch Number: 47
Batch Number: 48
Batch Number: 49
Batch Number: 50
Batch Number: 51
Batch Number: 52
Batch Number: 53
Batch Number: 54


---

### Other

In [ ]:
try:
    with open(range_dict_path, 'r') as file:
        range_dict = json.load(file)
except:
    print('No float range dictionary found!')
quant_layers_list = list(range_dict.keys())
quant_layers_list
tmp_max = range_dict[quant_layers_list[i]]['max']

In [ ]:
class QuantizedInput(tf.keras.layers.Layer):
    def __init__(self, min_val=-1.0, max_val=1.0, **kwargs):
        super().__init__(**kwargs)
        self.min_val = min_val
        self.max_val = max_val

    def call(self, inputs):
        return tf.quantization.fake_quant_with_min_max_vars(
            inputs, min=self.min_val, max=self.max_val, num_bits=8)

In [ ]:
def fake_quant_layer(x, min_val=-6.0, max_val=6.0):
    return tf.quantization.fake_quant_with_min_max_vars(x, min=min_val, max=max_val)

In [ ]:
new_model = Sequential()
for layer in model.layers:
    cloned_layer = tf.keras.models.clone_model(layer)
    cloned_layer.build(layer.input_shape)
    cloned_layer.set_weights(layer.get_weights())
    new_model.add(cloned_layer)

    if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
        new_model.add(QuantizedInput())

In [ ]:
# # Create new model
# new_model = Sequential()

# # Insert layers from the old model one by one
# i = 0
# for layer in model.layers:
#     # Insert new layer after the first Dense
#     if isinstance(layer, Conv2D) or isinstance(layer, Dense):
#         tmp_max = range_dict[quant_layers_list[i]]['max']
#         new_model.add(QuantizedInput(min_val=0, max_val=tmp_max))
#         new_model.add(layer)
#         i = i + 1

# if i != len(quant_layers_list):
#     print("Wrong layer handling!")

In [ ]:
# new_model_layers_list = []
# for i in new_model.layers:
#     new_model_layers_list.append(i.name)

# i = 0
# for layer in model.layers:
#     if layer == new_model_layers_list[i]:
#         print(f"Copying {layer.name} weights!")
#         new_layer = new_model.layers[new_model_layers_list[i]]
#         new_layer.set_weights(layer.get_weights())
#         i = i + 1

### Evaluate new model with quantized activations